In [4]:
import os
import json
import logging
import re
import pandas as pd
from datetime import datetime
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from base64 import urlsafe_b64decode
import pytz

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def auto_authenticate_google():
    """Authenticates with Gmail API using OAuth 2.0"""
    print("\n=== Starting Authentication ===")
    creds = None
    token_path = 'token.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        print("Found existing token file, loading credentials...")
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                print("Credentials expired, refreshing...")
                creds.refresh(Request())
        except Exception as e:
            print(f"Error loading credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        print("No valid credentials found, initiating OAuth flow...")
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client.json', SCOPES)
            creds = flow.run_local_server(port=8080)
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
            print("Authentication successful! Token saved.")
        except Exception as e:
            print(f"Authentication failed: {e}")
            raise
    
    try:
        print("Building Gmail service...")
        gmail_service = build('gmail', 'v1', credentials=creds)
        print("Gmail service ready!")
        return gmail_service
    except Exception as e:
        print(f"Failed to build Gmail service: {e}")
        raise

def get_recent_emails(service, max_results=30):
    """Fetches the most recent emails from the inbox"""
    print(f"\n=== Fetching {max_results} most recent emails ===")
    try:
        results = service.users().messages().list(
            userId="me",
            labelIds=['INBOX'],
            maxResults=max_results
        ).execute()
        messages = results.get('messages', [])
        print(f"Found {len(messages)} emails in inbox")
        return messages
    except Exception as e:
        print(f"Error fetching emails: {e}")
        return []

def decode_base64(data):
    """Decodes base64 email content with proper padding"""
    missing_padding = len(data) % 4
    if missing_padding:
        data += '=' * (4 - missing_padding)
    return urlsafe_b64decode(data)

def extract_email_body(service, message_id):
    """Extracts and decodes the email body content"""
    try:
        msg = service.users().messages().get(
            userId="me",
            id=message_id,
            format='full'
        ).execute()
        
        payload = msg.get('payload', {})
        
        # Extract body from payload
        body = ""
        if 'body' in payload and 'data' in payload['body']:
            body = decode_base64(payload['body']['data']).decode('utf-8', errors='ignore')
        elif 'parts' in payload:
            for part in payload['parts']:
                if part.get('mimeType') in ['text/plain', 'text/html']:
                    if 'body' in part and 'data' in part['body']:
                        body = decode_base64(part['body']['data']).decode('utf-8', errors='ignore')
                        break
                elif 'parts' in part:
                    for subpart in part['parts']:
                        if subpart.get('mimeType') in ['text/plain', 'text/html']:
                            if 'body' in subpart and 'data' in subpart['body']:
                                body = decode_base64(subpart['body']['data']).decode('utf-8', errors='ignore')
                                break
        
        return body if body else None
    except Exception as e:
        print(f"Error processing email {message_id}: {e}")
        return None

def extract_job_details(body):
    """Enhanced version focusing on extracting dates from parentheses after Job IDs"""
    job_data = {
        'Title': None,
        'Job_ID': None,
        'Due_date': None
    }
    
    if not body:
        return job_data
    
    # Updated Title extraction pattern to catch:
    # 1. Hybrid/Local, Remote/Local, Onsite/Local (original pattern)
    # 2. Standalone Remote, Hybrid, Onsite (new addition)
    # 3. Followed by optional text until next pattern or end
    title_pattern = r'(?i)((?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?[^(]*\(.*?\)|(?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?.*?)(?=\s+with\s+|\s*\(|$|\n)'
    title_match = re.search(title_pattern, body)
    if title_match:
        job_data['Title'] = title_match.group(1).strip()
    
    # Rest of the function remains the same...
    # Job ID extraction (unchanged)
    id_pattern = r'(?i)(?:job\s*|req\s*|position\s*)?(?:id\s*|#\s*|num\s*|number\s*)?[:#]?\s*([A-Za-z]{2,}-?\d{3,})'
    id_match = re.search(id_pattern, body)
    if id_match:
        job_id = id_match.group(1).strip()
        job_data['Job_ID'] = job_id
        
        # === NEW: Focused date extraction from parentheses after Job ID ===
        # Looks for pattern: Job ID followed by parentheses with digits
        date_match = re.search(
            r'{}\s*\((\d+)\)'.format(re.escape(job_id)),
            body
        )
        if date_match:
            # Extract the last 4 digits and format as MM/DD
            full_number = date_match.group(1)
            last_four = full_number[-4:]  # Always take last 4 digits
            if len(last_four) == 4:  # Ensure we have exactly 4 digits
                job_data['Due_date'] = f"{last_four[:2]}/{last_four[2:]}"
    
    # Fallback: If no Job ID found, try standalone ID pattern (unchanged)
    else:
        standalone_id = r'(?<!\w)([A-Za-z]{2,}-?\d{3,})(?!\w)'
        id_match = re.search(standalone_id, body)
        if id_match:
            job_data['Job_ID'] = id_match.group(1).strip()
    
    # Fallback: Check for standalone parentheses dates if not found after Job ID
    if not job_data['Due_date']:
        standalone_date_match = re.search(r'\((\d+)\)', body)
        if standalone_date_match:
            full_number = standalone_date_match.group(1)
            last_four = full_number[-4:]  # Take last 4 digits
            if len(last_four) == 4:
                job_data['Due_date'] = f"{last_four[:2]}/{last_four[2:]}"
    
    return job_data

def main():
    """Main function to process emails and extract job details"""
    try:
        # Initialize results table
        results = []
        
        # Step 1: Authenticate
        service = auto_authenticate_google()
        
        # Step 2: Get recent emails
        messages = get_recent_emails(service, max_results=25)
        
        if not messages:
            print("No emails found in inbox")
            return
        
        # Step 3: Process each email
        print("\n=== Processing Emails ===")
        for i, message in enumerate(messages, 1):
            message_id = message['id']
            print(f"\nProcessing Email #{i} (ID: {message_id})")
            
            # Step 4: Extract body
            body = extract_email_body(service, message_id)
            
            if body:
                # Step 5: Extract job details
                job_details = extract_job_details(body)
                job_details['Email ID'] = message_id
                results.append(job_details)
                
                # Print found details
                print(f"Title: {job_details['Title']}")
                print(f"Job_ID: {job_details['Job_ID']}")
                print(f"Due_date: {job_details['Due_date']}")
            else:
                print("No body content could be extracted")
        
        # Create and display results table
        df = pd.DataFrame(results)
        if not df.empty:
            print("\n=== Final Results ===")
            print(df[['Email ID', 'Title', 'Job_ID', 'Due_date']].to_string(index=False))
            
            # Save to CSV
            df.to_csv('extracted_job_details.csv', index=False)
            print("\nResults saved to 'extracted_job_details.csv'")
        else:
            print("\nNo job details found in any emails")
            
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


=== Starting Authentication ===
Found existing token file, loading credentials...
Credentials expired, refreshing...


2025-07-22 12:41:12,564 - INFO - file_cache is only supported with oauth2client<4.0.0


Building Gmail service...
Gmail service ready!

=== Fetching 25 most recent emails ===
Found 8 emails in inbox

=== Processing Emails ===

Processing Email #1 (ID: 1982174cf77fb3a3)
Title: Hybrid/Local Analytics Engineer (MS Fabric/12+)
Job_ID: GA-768025
Due_date: 07/16

Processing Email #2 (ID: 198217128c79c421)
Title: Hybrid/Local Privacy Officer/Security Analyst (CISA/CIPM/CIPP)
Job_ID: NC-768095
Due_date: 07/24

Processing Email #3 (ID: 198216d126f86664)
Title: Remote Sitefinity Developer (12+)
Job_ID: NC-765767
Due_date: 06/25

Processing Email #4 (ID: 198216c4da407075)
Title: Remote Sitefinity Developer
Job_ID: NC-765767
Due_date: 07/21

Processing Email #5 (ID: 1982169418497b56)
Title: Remote/Local GIS Analyst (12+)
Job_ID: NC-767681
Due_date: 07/18

Processing Email #6 (ID: 198216585c548259)
Title: Hybrid/Local (Lansing ONLY) .NET Developer (12+)
Job_ID: MI-144480
Due_date: 07/21

Processing Email #7 (ID: 1982135e4ab36a6a)
Title: Hybrid/Local (Lansing 90 min) Full-stack Java De

In [1]:
import os
import csv
import json
import logging
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from base64 import urlsafe_b64decode
import pytz
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def auto_authenticate_google():
    """Automatically authenticates with Google APIs."""
    creds = None
    token_path = 'token1.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                creds.refresh(Request())
        except Exception as e:
            logging.error(f"Error loading credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client1.json', SCOPES)
            creds = flow.run_local_server(port=8080)
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
        except Exception as e:
            logging.error(f"Authentication failed: {e}")
            raise
    
    try:
        gmail_service = build('gmail', 'v1', credentials=creds)
        return gmail_service
    except Exception as e:
        logging.error(f"Failed to build services: {e}")
        raise

def count_emails_for_job_id(service, job_id):
    """Count emails containing the job ID in the inbox."""
    try:
        query = f'"{job_id}"'  # Using quotes for exact matching
        results = service.users().messages().list(
            userId="me",
            q=query,
            labelIds=['INBOX']
        ).execute()
        return results.get('resultSizeEstimate', 0)
    except Exception as e:
        logging.error(f"Error counting emails for {job_id}: {e}")
        return 0

def process_job_ids(csv_path, service):
    """Process job IDs from CSV and update with email counts."""
    # Read the existing CSV file
    try:
        with open(csv_path, 'r') as file:
            reader = csv.DictReader(file)
            rows = list(reader)
            fieldnames = reader.fieldnames
            
        # Add 'No_of_emails' column if it doesn't exist
        if 'No_of_emails' not in fieldnames:
            fieldnames.append('No_of_emails')
            for row in rows:
                row['No_of_emails'] = '0'
    
        # Process each job ID
        for row in rows:
            job_id = row.get('Job ID', '').strip() or row.get('Job_ID', '').strip()
            if job_id:  # Only process if job_id exists
                logging.info(f"Searching for emails with job ID: {job_id}")
                email_count = count_emails_for_job_id(service, job_id)
                row['No_of_emails'] = str(email_count)
                logging.info(f"Found {email_count} emails for {job_id}")
                time.sleep(0.5)  # Add small delay to avoid rate limiting
    
        # Write back to the CSV file
        with open(csv_path, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
    
        logging.info(f"Successfully updated CSV file: {csv_path}")
        
    except Exception as e:
        logging.error(f"Error processing CSV file: {e}")
        raise

def main():
    try:
        # Authenticate with Gmail API
        service = auto_authenticate_google()
        
        # Path to your existing CSV file
        csv_path = 'extracted_job_details.csv'
        
        # Process the job IDs and update the CSV
        process_job_ids(csv_path, service)
        
    except Exception as e:
        logging.error(f"Error in main execution: {e}")

if __name__ == "__main__":
    main()

2025-07-22 11:39:02,438 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-07-22 11:39:02,454 - INFO - Searching for emails with job ID: GA-768025
2025-07-22 11:39:03,677 - INFO - Found 3 emails for GA-768025
2025-07-22 11:39:04,178 - INFO - Searching for emails with job ID: NC-768095
2025-07-22 11:39:04,639 - INFO - Found 7 emails for NC-768095
2025-07-22 11:39:05,154 - INFO - Searching for emails with job ID: NC-765767
2025-07-22 11:39:05,622 - INFO - Found 6 emails for NC-765767
2025-07-22 11:39:06,128 - INFO - Searching for emails with job ID: NC-765767
2025-07-22 11:39:06,589 - INFO - Found 6 emails for NC-765767
2025-07-22 11:39:07,093 - INFO - Searching for emails with job ID: NC-767681
2025-07-22 11:39:08,708 - INFO - Found 6 emails for NC-767681
2025-07-22 11:39:09,223 - INFO - Searching for emails with job ID: MI-144480
2025-07-22 11:39:11,146 - INFO - Found 11 emails for MI-144480
2025-07-22 11:39:11,658 - INFO - Searching for emails with job ID: MI-144516
20

In [3]:
import os
import json
import logging
import re
import pandas as pd
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from base64 import urlsafe_b64decode
import pytz
import csv
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def auto_authenticate_primary_gmail():
    """Authenticates with primary Gmail account (token.json)"""
    print("\n=== Authenticating Primary Gmail Account ===")
    creds = None
    token_path = 'token.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        print("Found primary token file, loading credentials...")
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                print("Primary credentials expired, refreshing...")
                creds.refresh(Request())
        except Exception as e:
            print(f"Error loading primary credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        print("No valid primary credentials found, initiating OAuth flow...")
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client.json', SCOPES)
            creds = flow.run_local_server(port=8080)
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
            print("Primary authentication successful! Token saved.")
        except Exception as e:
            print(f"Primary authentication failed: {e}")
            raise
    
    try:
        print("Building primary Gmail service...")
        gmail_service = build('gmail', 'v1', credentials=creds)
        print("Primary Gmail service ready!")
        return gmail_service
    except Exception as e:
        print(f"Failed to build primary Gmail service: {e}")
        raise

def auto_authenticate_secondary_gmail():
    """Authenticates with secondary Gmail account (token1.json)"""
    print("\n=== Authenticating Secondary Gmail Account ===")
    creds = None
    token_path = 'token1.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        print("Found secondary token file, loading credentials...")
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                print("Secondary credentials expired, refreshing...")
                creds.refresh(Request())
        except Exception as e:
            print(f"Error loading secondary credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        print("No valid secondary credentials found, initiating OAuth flow...")
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client1.json', SCOPES)
            creds = flow.run_local_server(port=8081)  # Different port than primary
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
            print("Secondary authentication successful! Token saved.")
        except Exception as e:
            print(f"Secondary authentication failed: {e}")
            raise
    
    try:
        print("Building secondary Gmail service...")
        gmail_service = build('gmail', 'v1', credentials=creds)
        print("Secondary Gmail service ready!")
        return gmail_service
    except Exception as e:
        print(f"Failed to build secondary Gmail service: {e}")
        raise

def get_recent_emails(service, max_results=30):
    """Fetches the most recent emails from the inbox"""
    print(f"\n=== Fetching {max_results} most recent emails ===")
    try:
        results = service.users().messages().list(
            userId="me",
            labelIds=['INBOX'],
            maxResults=max_results
        ).execute()
        messages = results.get('messages', [])
        print(f"Found {len(messages)} emails in inbox")
        return messages
    except Exception as e:
        print(f"Error fetching emails: {e}")
        return []

def decode_base64(data):
    """Decodes base64 email content with proper padding"""
    missing_padding = len(data) % 4
    if missing_padding:
        data += '=' * (4 - missing_padding)
    return urlsafe_b64decode(data)

def extract_email_body(service, message_id):
    """Extracts and decodes the email body content"""
    try:
        msg = service.users().messages().get(
            userId="me",
            id=message_id,
            format='full'
        ).execute()
        
        payload = msg.get('payload', {})
        
        # Extract body from payload
        body = ""
        if 'body' in payload and 'data' in payload['body']:
            body = decode_base64(payload['body']['data']).decode('utf-8', errors='ignore')
        elif 'parts' in payload:
            for part in payload['parts']:
                if part.get('mimeType') in ['text/plain', 'text/html']:
                    if 'body' in part and 'data' in part['body']:
                        body = decode_base64(part['body']['data']).decode('utf-8', errors='ignore')
                        break
                elif 'parts' in part:
                    for subpart in part['parts']:
                        if subpart.get('mimeType') in ['text/plain', 'text/html']:
                            if 'body' in subpart and 'data' in subpart['body']:
                                body = decode_base64(subpart['body']['data']).decode('utf-8', errors='ignore')
                                break
        
        return body if body else None
    except Exception as e:
        print(f"Error processing email {message_id}: {e}")
        return None

def extract_job_details(body):
    """Enhanced version focusing on extracting dates from parentheses after Job IDs"""
    job_data = {
        'Title': None,
        'Job_ID': None,
        'Due_date': None
    }
    
    if not body:
        return job_data
    
    # Title extraction pattern
    title_pattern = r'(?i)((?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?[^(]*\(.*?\)|(?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?.*?)(?=\s+with\s+|\s*\(|$|\n)'
    title_match = re.search(title_pattern, body)
    if title_match:
        job_data['Title'] = title_match.group(1).strip()
    
    # Job ID extraction
    id_pattern = r'(?i)(?:job\s*|req\s*|position\s*)?(?:id\s*|#\s*|num\s*|number\s*)?[:#]?\s*([A-Za-z]{2,}-?\d{3,})'
    id_match = re.search(id_pattern, body)
    if id_match:
        job_id = id_match.group(1).strip()
        job_data['Job_ID'] = job_id
        
        # Date extraction from parentheses after Job ID
        date_match = re.search(
            r'{}\s*\((\d+)\)'.format(re.escape(job_id)),
            body
        )
        if date_match:
            full_number = date_match.group(1)
            last_four = full_number[-4:]
            if len(last_four) == 4:
                job_data['Due_date'] = f"{last_four[:2]}/{last_four[2:]}"
    
    # Fallback: If no Job ID found, try standalone ID pattern
    else:
        standalone_id = r'(?<!\w)([A-Za-z]{2,}-?\d{3,})(?!\w)'
        id_match = re.search(standalone_id, body)
        if id_match:
            job_data['Job_ID'] = id_match.group(1).strip()
    
    # Fallback: Check for standalone parentheses dates
    if not job_data['Due_date']:
        standalone_date_match = re.search(r'\((\d+)\)', body)
        if standalone_date_match:
            full_number = standalone_date_match.group(1)
            last_four = full_number[-4:]
            if len(last_four) == 4:
                job_data['Due_date'] = f"{last_four[:2]}/{last_four[2:]}"
    
    return job_data

def count_emails_for_job_id(service, job_id):
    """Count emails containing the job ID in the inbox."""
    try:
        query = f'"{job_id}"'  # Using quotes for exact matching
        results = service.users().messages().list(
            userId="me",
            q=query,
            labelIds=['INBOX']
        ).execute()
        return results.get('resultSizeEstimate', 0)
    except Exception as e:
        logging.error(f"Error counting emails for {job_id}: {e}")
        return 0

def process_job_ids(csv_path, service):
    """Process job IDs from CSV and update with email counts."""
    try:
        with open(csv_path, 'r') as file:
            reader = csv.DictReader(file)
            rows = list(reader)
            fieldnames = reader.fieldnames
            
        if 'No_of_emails' not in fieldnames:
            fieldnames.append('No_of_emails')
            for row in rows:
                row['No_of_emails'] = '0'
    
        for row in rows:
            job_id = row.get('Job ID', '').strip() or row.get('Job_ID', '').strip()
            if job_id:
                logging.info(f"Searching for emails with job ID: {job_id}")
                email_count = count_emails_for_job_id(service, job_id)
                row['No_of_emails'] = str(email_count)
                logging.info(f"Found {email_count} emails for {job_id}")
                time.sleep(0.5)  # Rate limiting
    
        with open(csv_path, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
    
        logging.info(f"Successfully updated CSV file: {csv_path}")
        
    except Exception as e:
        logging.error(f"Error processing CSV file: {e}")
        raise

def main():
    """Main function to process both Gmail accounts"""
    try:
        # Initialize results table
        results = []
        
        # Step 1: Authenticate with primary Gmail (extraction account)
        primary_service = auto_authenticate_primary_gmail()
        
        # Step 2: Get recent emails from primary account
        messages = get_recent_emails(primary_service, max_results=25)
        
        if not messages:
            print("No emails found in primary inbox")
            return
        
        # Step 3: Process each email from primary account
        print("\n=== Processing Emails from Primary Account ===")
        for i, message in enumerate(messages, 1):
            message_id = message['id']
            print(f"\nProcessing Email #{i} (ID: {message_id})")
            
            body = extract_email_body(primary_service, message_id)
            
            if body:
                job_details = extract_job_details(body)
                job_details['Email ID'] = message_id
                results.append(job_details)
                
                print(f"Title: {job_details['Title']}")
                print(f"Job_ID: {job_details['Job_ID']}")
                print(f"Due_date: {job_details['Due_date']}")
            else:
                print("No body content could be extracted")
        
        # Create and save results table
        df = pd.DataFrame(results)
        if not df.empty:
            print("\n=== Final Results ===")
            print(df[['Email ID', 'Title', 'Job_ID', 'Due_date']].to_string(index=False))
            
            csv_path = 'extracted_job_details.csv'
            df.to_csv(csv_path, index=False)
            print(f"\nResults saved to '{csv_path}'")
            
            # Step 4: Authenticate with secondary Gmail (counting account)
            secondary_service = auto_authenticate_secondary_gmail()
            
            # Step 5: Process CSV with secondary account
            process_job_ids(csv_path, secondary_service)
            print("\nAdded email counts from secondary account to the CSV file")
        else:
            print("\nNo job details found in any emails")
            
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


=== Authenticating Primary Gmail Account ===
Found primary token file, loading credentials...
Primary credentials expired, refreshing...


2025-07-22 15:36:17,479 - INFO - file_cache is only supported with oauth2client<4.0.0


Building primary Gmail service...
Primary Gmail service ready!

=== Fetching 25 most recent emails ===
Found 8 emails in inbox

=== Processing Emails from Primary Account ===

Processing Email #1 (ID: 1982174cf77fb3a3)
Error processing email 1982174cf77fb3a3: The read operation timed out
No body content could be extracted

Processing Email #2 (ID: 198217128c79c421)
Title: Hybrid/Local Privacy Officer/Security Analyst (CISA/CIPM/CIPP)
Job_ID: NC-768095
Due_date: 07/24

Processing Email #3 (ID: 198216d126f86664)
Title: Remote Sitefinity Developer (12+)
Job_ID: NC-765767
Due_date: 06/25

Processing Email #4 (ID: 198216c4da407075)
Title: Remote Sitefinity Developer
Job_ID: NC-765767
Due_date: 07/21

Processing Email #5 (ID: 1982169418497b56)
Title: Remote/Local GIS Analyst (12+)
Job_ID: NC-767681
Due_date: 07/18

Processing Email #6 (ID: 198216585c548259)
Title: Hybrid/Local (Lansing ONLY) .NET Developer (12+)
Job_ID: MI-144480
Due_date: 07/21

Processing Email #7 (ID: 1982135e4ab36a6a)
Ti

2025-07-22 15:37:37,743 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-07-22 15:37:37,766 - INFO - Searching for emails with job ID: NC-768095


Building secondary Gmail service...
Secondary Gmail service ready!


2025-07-22 15:37:38,898 - INFO - Found 7 emails for NC-768095
2025-07-22 15:37:39,419 - INFO - Searching for emails with job ID: NC-765767
2025-07-22 15:37:39,909 - INFO - Found 6 emails for NC-765767
2025-07-22 15:37:40,416 - INFO - Searching for emails with job ID: NC-765767
2025-07-22 15:37:40,889 - INFO - Found 6 emails for NC-765767
2025-07-22 15:37:41,397 - INFO - Searching for emails with job ID: NC-767681
2025-07-22 15:37:42,155 - INFO - Found 6 emails for NC-767681
2025-07-22 15:37:42,664 - INFO - Searching for emails with job ID: MI-144480
2025-07-22 15:37:43,130 - INFO - Found 11 emails for MI-144480
2025-07-22 15:37:43,638 - INFO - Searching for emails with job ID: MI-144516
2025-07-22 15:37:44,365 - INFO - Found 1 emails for MI-144516
2025-07-22 15:37:44,880 - INFO - Searching for emails with job ID: VA-767445
2025-07-22 15:38:19,416 - INFO - Found 0 emails for VA-767445
2025-07-22 15:38:19,944 - INFO - Successfully updated CSV file: extracted_job_details.csv



Added email counts from secondary account to the CSV file


In [ ]:
import os
import json
import logging
import re
import pandas as pd
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from base64 import urlsafe_b64decode
import pytz
import csv
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def auto_authenticate_primary_gmail():
    """Authenticates with primary Gmail account (token.json)"""
    print("\n=== Authenticating Primary Gmail Account ===")
    creds = None
    token_path = 'token.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        print("Found primary token file, loading credentials...")
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                print("Primary credentials expired, refreshing...")
                creds.refresh(Request())
        except Exception as e:
            print(f"Error loading primary credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        print("No valid primary credentials found, initiating OAuth flow...")
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client.json', SCOPES)
            creds = flow.run_local_server(port=8080)
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
            print("Primary authentication successful! Token saved.")
        except Exception as e:
            print(f"Primary authentication failed: {e}")
            raise
    
    try:
        print("Building primary Gmail service...")
        gmail_service = build('gmail', 'v1', credentials=creds)
        print("Primary Gmail service ready!")
        return gmail_service
    except Exception as e:
        print(f"Failed to build primary Gmail service: {e}")
        raise

def auto_authenticate_secondary_gmail():
    """Authenticates with secondary Gmail account (token1.json)"""
    print("\n=== Authenticating Secondary Gmail Account ===")
    creds = None
    token_path = 'token1.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        print("Found secondary token file, loading credentials...")
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                print("Secondary credentials expired, refreshing...")
                creds.refresh(Request())
        except Exception as e:
            print(f"Error loading secondary credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        print("No valid secondary credentials found, initiating OAuth flow...")
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client1.json', SCOPES)
            creds = flow.run_local_server(port=8081)  # Different port than primary
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
            print("Secondary authentication successful! Token saved.")
        except Exception as e:
            print(f"Secondary authentication failed: {e}")
            raise
    
    try:
        print("Building secondary Gmail service...")
        gmail_service = build('gmail', 'v1', credentials=creds)
        print("Secondary Gmail service ready!")
        return gmail_service
    except Exception as e:
        print(f"Failed to build secondary Gmail service: {e}")
        raise

def get_recent_emails(service, max_results=30):
    """Fetches the most recent emails from the inbox"""
    print(f"\n=== Fetching {max_results} most recent emails ===")
    try:
        results = service.users().messages().list(
            userId="me",
            labelIds=['INBOX'],
            maxResults=max_results
        ).execute()
        messages = results.get('messages', [])
        print(f"Found {len(messages)} emails in inbox")
        return messages
    except Exception as e:
        print(f"Error fetching emails: {e}")
        return []

def decode_base64(data):
    """Decodes base64 email content with proper padding"""
    missing_padding = len(data) % 4
    if missing_padding:
        data += '=' * (4 - missing_padding)
    return urlsafe_b64decode(data)

def extract_email_body(service, message_id):
    """Extracts and decodes the email body content"""
    try:
        msg = service.users().messages().get(
            userId="me",
            id=message_id,
            format='full'
        ).execute()
        
        payload = msg.get('payload', {})
        
        # Extract body from payload
        body = ""
        if 'body' in payload and 'data' in payload['body']:
            body = decode_base64(payload['body']['data']).decode('utf-8', errors='ignore')
        elif 'parts' in payload:
            for part in payload['parts']:
                if part.get('mimeType') in ['text/plain', 'text/html']:
                    if 'body' in part and 'data' in part['body']:
                        body = decode_base64(part['body']['data']).decode('utf-8', errors='ignore')
                        break
                elif 'parts' in part:
                    for subpart in part['parts']:
                        if subpart.get('mimeType') in ['text/plain', 'text/html']:
                            if 'body' in subpart and 'data' in subpart['body']:
                                body = decode_base64(subpart['body']['data']).decode('utf-8', errors='ignore')
                                break
        
        return body if body else None
    except Exception as e:
        print(f"Error processing email {message_id}: {e}")
        return None

def extract_job_details(body):
    """Enhanced version focusing on extracting dates from parentheses after Job IDs"""
    job_data = {
        'Title': None,
        'Job_ID': None,
        'Due_date': None
    }
    
    if not body:
        return job_data
    
    # Title extraction pattern
    title_pattern = r'(?i)((?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?[^(]*\(.*?\)|(?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?.*?)(?=\s+with\s+|\s*\(|$|\n)'
    title_match = re.search(title_pattern, body)
    if title_match:
        job_data['Title'] = title_match.group(1).strip()
    
    # Job ID extraction
    id_pattern = r'(?i)(?:job\s*|req\s*|position\s*)?(?:id\s*|#\s*|num\s*|number\s*)?[:#]?\s*([A-Za-z]{2,}-?\d{3,})'
    id_match = re.search(id_pattern, body)
    if id_match:
        job_id = id_match.group(1).strip()
        job_data['Job_ID'] = job_id
        
        # Date extraction from parentheses after Job ID
        date_match = re.search(
            r'{}\s*\((\d+)\)'.format(re.escape(job_id)),
            body
        )
        if date_match:
            full_number = date_match.group(1)
            last_four = full_number[-4:]
            if len(last_four) == 4:
                job_data['Due_date'] = f"{last_four[:2]}/{last_four[2:]}"
    
    # Fallback: If no Job ID found, try standalone ID pattern
    else:
        standalone_id = r'(?<!\w)([A-Za-z]{2,}-?\d{3,})(?!\w)'
        id_match = re.search(standalone_id, body)
        if id_match:
            job_data['Job_ID'] = id_match.group(1).strip()
    
    # Fallback: Check for standalone parentheses dates
    if not job_data['Due_date']:
        standalone_date_match = re.search(r'\((\d+)\)', body)
        if standalone_date_match:
            full_number = standalone_date_match.group(1)
            last_four = full_number[-4:]
            if len(last_four) == 4:
                job_data['Due_date'] = f"{last_four[:2]}/{last_four[2:]}"
    
    return job_data

def count_emails_for_job_id(service, job_id):
    """Count emails containing the job ID in the inbox."""
    try:
        query = f'"{job_id}"'  # Using quotes for exact matching
        results = service.users().messages().list(
            userId="me",
            q=query,
            labelIds=['INBOX']
        ).execute()
        return results.get('resultSizeEstimate', 0)
    except Exception as e:
        logging.error(f"Error counting emails for {job_id}: {e}")
        return 0

def process_job_ids(csv_path, service):
    """Process job IDs from CSV and update with email counts."""
    try:
        with open(csv_path, 'r') as file:
            reader = csv.DictReader(file)
            rows = list(reader)
            fieldnames = reader.fieldnames
            
        if 'No_of_emails' not in fieldnames:
            fieldnames.append('No_of_emails')
            for row in rows:
                row['No_of_emails'] = '0'
    
        for row in rows:
            job_id = row.get('Job ID', '').strip() or row.get('Job_ID', '').strip()
            if job_id:
                logging.info(f"Searching for emails with job ID: {job_id}")
                email_count = count_emails_for_job_id(service, job_id)
                row['No_of_emails'] = str(email_count)
                logging.info(f"Found {email_count} emails for {job_id}")
                time.sleep(0.5)  # Rate limiting
    
        with open(csv_path, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
    
        logging.info(f"Successfully updated CSV file: {csv_path}")
        
    except Exception as e:
        logging.error(f"Error processing CSV file: {e}")
        raise

def main():
    """Main function to process both Gmail accounts"""
    try:
        # Initialize results table
        results = []
        
        # Step 1: Authenticate with primary Gmail (extraction account)
        primary_service = auto_authenticate_primary_gmail()
        
        # Step 2: Get recent emails from primary account
        messages = get_recent_emails(primary_service, max_results=25)
        
        if not messages:
            print("No emails found in primary inbox")
            return
        
        # Step 3: Process each email from primary account
        print("\n=== Processing Emails from Primary Account ===")
        for i, message in enumerate(messages, 1):
            message_id = message['id']
            print(f"\nProcessing Email #{i} (ID: {message_id})")
            
            body = extract_email_body(primary_service, message_id)
            
            if body:
                job_details = extract_job_details(body)
                job_details['Email ID'] = message_id
                results.append(job_details)
                
                print(f"Title: {job_details['Title']}")
                print(f"Job_ID: {job_details['Job_ID']}")
                print(f"Due_date: {job_details['Due_date']}")
            else:
                print("No body content could be extracted")
        
        # Create and save results table
        df = pd.DataFrame(results)
        if not df.empty:
            print("\n=== Final Results ===")
            print(df[['Email ID', 'Title', 'Job_ID', 'Due_date']].to_string(index=False))
            
            csv_path = 'duedates.csv'
            df.to_csv(csv_path, index=False)
            print(f"\nResults saved to '{csv_path}'")
            
            # Step 4: Authenticate with secondary Gmail (counting account)
            secondary_service = auto_authenticate_secondary_gmail()
            
            # Step 5: Process CSV with secondary account
            process_job_ids(csv_path, secondary_service)
            print("\nAdded email counts from secondary account to the CSV file")
        else:
            print("\nNo job details found in any emails")
            
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


=== Authenticating Primary Gmail Account ===
Found primary token file, loading credentials...
Primary credentials expired, refreshing...


2025-07-22 16:21:55,532 - INFO - file_cache is only supported with oauth2client<4.0.0


Building primary Gmail service...
Primary Gmail service ready!

=== Fetching 25 most recent emails ===
Found 8 emails in inbox

=== Processing Emails from Primary Account ===

Processing Email #1 (ID: 1982174cf77fb3a3)
Title: Hybrid/Local Analytics Engineer (MS Fabric/12+)
Job_ID: GA-768025
Due_date: 07/16

Processing Email #2 (ID: 198217128c79c421)
Title: Hybrid/Local Privacy Officer/Security Analyst (CISA/CIPM/CIPP)
Job_ID: NC-768095
Due_date: 07/24

Processing Email #3 (ID: 198216d126f86664)
Title: Remote Sitefinity Developer (12+)
Job_ID: NC-765767
Due_date: 06/25

Processing Email #4 (ID: 198216c4da407075)
Title: Remote Sitefinity Developer
Job_ID: NC-765767
Due_date: 07/21

Processing Email #5 (ID: 1982169418497b56)
Title: Remote/Local GIS Analyst (12+)
Job_ID: NC-767681
Due_date: 07/18

Processing Email #6 (ID: 198216585c548259)
Title: Hybrid/Local (Lansing ONLY) .NET Developer (12+)
Job_ID: MI-144480
Due_date: 07/21

Processing Email #7 (ID: 1982135e4ab36a6a)
Title: Hybrid/Loca

2025-07-22 16:22:00,190 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-07-22 16:22:00,207 - INFO - Searching for emails with job ID: GA-768025


Building secondary Gmail service...
Secondary Gmail service ready!


2025-07-22 16:22:00,784 - INFO - Found 3 emails for GA-768025
2025-07-22 16:22:01,285 - INFO - Searching for emails with job ID: NC-768095
2025-07-22 16:22:01,719 - INFO - Found 7 emails for NC-768095
2025-07-22 16:22:02,232 - INFO - Searching for emails with job ID: NC-765767
2025-07-22 16:22:02,664 - INFO - Found 6 emails for NC-765767
2025-07-22 16:22:03,170 - INFO - Searching for emails with job ID: NC-765767
2025-07-22 16:22:03,617 - INFO - Found 6 emails for NC-765767
2025-07-22 16:22:04,119 - INFO - Searching for emails with job ID: NC-767681
2025-07-22 16:22:04,549 - INFO - Found 6 emails for NC-767681
2025-07-22 16:22:05,069 - INFO - Searching for emails with job ID: MI-144480
2025-07-22 16:22:05,519 - INFO - Found 11 emails for MI-144480
2025-07-22 16:22:06,034 - INFO - Searching for emails with job ID: MI-144516
2025-07-22 16:22:06,467 - INFO - Found 1 emails for MI-144516
2025-07-22 16:22:06,983 - INFO - Searching for emails with job ID: VA-767445
2025-07-22 16:22:07,418 - 


Added email counts from secondary account to the CSV file


In [8]:
import os
import json
import logging
import re
import pandas as pd
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from base64 import urlsafe_b64decode
import pytz
import csv
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def auto_authenticate_primary_gmail():
    """Authenticates with primary Gmail account (token.json)"""
    print("\n=== Authenticating Primary Gmail Account ===")
    creds = None
    token_path = 'token.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        print("Found primary token file, loading credentials...")
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                print("Primary credentials expired, refreshing...")
                creds.refresh(Request())
        except Exception as e:
            print(f"Error loading primary credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        print("No valid primary credentials found, initiating OAuth flow...")
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client.json', SCOPES)
            creds = flow.run_local_server(port=8080)
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
            print("Primary authentication successful! Token saved.")
        except Exception as e:
            print(f"Primary authentication failed: {e}")
            raise
    
    try:
        print("Building primary Gmail service...")
        gmail_service = build('gmail', 'v1', credentials=creds)
        print("Primary Gmail service ready!")
        return gmail_service
    except Exception as e:
        print(f"Failed to build primary Gmail service: {e}")
        raise

def auto_authenticate_secondary_gmail():
    """Authenticates with secondary Gmail account (token1.json)"""
    print("\n=== Authenticating Secondary Gmail Account ===")
    creds = None
    token_path = 'token1.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        print("Found secondary token file, loading credentials...")
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                print("Secondary credentials expired, refreshing...")
                creds.refresh(Request())
        except Exception as e:
            print(f"Error loading secondary credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        print("No valid secondary credentials found, initiating OAuth flow...")
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client1.json', SCOPES)
            creds = flow.run_local_server(port=8081)  # Different port than primary
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
            print("Secondary authentication successful! Token saved.")
        except Exception as e:
            print(f"Secondary authentication failed: {e}")
            raise
    
    try:
        print("Building secondary Gmail service...")
        gmail_service = build('gmail', 'v1', credentials=creds)
        print("Secondary Gmail service ready!")
        return gmail_service
    except Exception as e:
        print(f"Failed to build secondary Gmail service: {e}")
        raise

def get_recent_emails(service, max_results=30):
    """Fetches the most recent emails from the inbox"""
    print(f"\n=== Fetching {max_results} most recent emails ===")
    try:
        results = service.users().messages().list(
            userId="me",
            labelIds=['INBOX'],
            maxResults=max_results
        ).execute()
        messages = results.get('messages', [])
        print(f"Found {len(messages)} emails in inbox")
        return messages
    except Exception as e:
        print(f"Error fetching emails: {e}")
        return []

def decode_base64(data):
    """Decodes base64 email content with proper padding"""
    missing_padding = len(data) % 4
    if missing_padding:
        data += '=' * (4 - missing_padding)
    return urlsafe_b64decode(data)

def extract_email_body(service, message_id):
    """Extracts and decodes the email body content"""
    try:
        msg = service.users().messages().get(
            userId="me",
            id=message_id,
            format='full'
        ).execute()
        
        payload = msg.get('payload', {})
        
        # Extract body from payload
        body = ""
        if 'body' in payload and 'data' in payload['body']:
            body = decode_base64(payload['body']['data']).decode('utf-8', errors='ignore')
        elif 'parts' in payload:
            for part in payload['parts']:
                if part.get('mimeType') in ['text/plain', 'text/html']:
                    if 'body' in part and 'data' in part['body']:
                        body = decode_base64(part['body']['data']).decode('utf-8', errors='ignore')
                        break
                elif 'parts' in part:
                    for subpart in part['parts']:
                        if subpart.get('mimeType') in ['text/plain', 'text/html']:
                            if 'body' in subpart and 'data' in subpart['body']:
                                body = decode_base64(subpart['body']['data']).decode('utf-8', errors='ignore')
                                break
        
        return body if body else None
    except Exception as e:
        print(f"Error processing email {message_id}: {e}")
        return None

def extract_job_details(body):
    """Enhanced version focusing on extracting dates from parentheses after Job IDs"""
    job_data = {
        'Title': None,
        'Job_ID': None,
        'Due_date': None
    }
    
    if not body:
        return job_data
    
    # Title extraction pattern
    title_pattern = r'(?i)((?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?[^(]*\(.*?\)|(?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?.*?)(?=\s+with\s+|\s*\(|$|\n)'
    title_match = re.search(title_pattern, body)
    if title_match:
        job_data['Title'] = title_match.group(1).strip()
    
    # Job ID extraction
    id_pattern = r'(?i)(?:job\s*|req\s*|position\s*)?(?:id\s*|#\s*|num\s*|number\s*)?[:#]?\s*([A-Za-z]{2,}-?\d{3,})'
    id_match = re.search(id_pattern, body)
    if id_match:
        job_id = id_match.group(1).strip()
        job_data['Job_ID'] = job_id
        
        # Date extraction from parentheses after Job ID
        date_match = re.search(
            r'{}\s*\((\d+)\)'.format(re.escape(job_id)),
            body
        )
        if date_match:
            full_number = date_match.group(1)
            last_four = full_number[-4:]
            if len(last_four) == 4:
                job_data['Due_date'] = f"{last_four[:2]}/{last_four[2:]}"
    
    # Fallback: If no Job ID found, try standalone ID pattern
    else:
        standalone_id = r'(?<!\w)([A-Za-z]{2,}-?\d{3,})(?!\w)'
        id_match = re.search(standalone_id, body)
        if id_match:
            job_data['Job_ID'] = id_match.group(1).strip()
    
    # Fallback: Check for standalone parentheses dates
    if not job_data['Due_date']:
        standalone_date_match = re.search(r'\((\d+)\)', body)
        if standalone_date_match:
            full_number = standalone_date_match.group(1)
            last_four = full_number[-4:]
            if len(last_four) == 4:
                job_data['Due_date'] = f"{last_four[:2]}/{last_four[2:]}"
    
    return job_data

def count_emails_for_job_id(service, job_id):
    """Count emails containing the job ID in the inbox."""
    try:
        query = f'"{job_id}"'  # Using quotes for exact matching
        results = service.users().messages().list(
            userId="me",
            q=query,
            labelIds=['INBOX']
        ).execute()
        return results.get('resultSizeEstimate', 0)
    except Exception as e:
        logging.error(f"Error counting emails for {job_id}: {e}")
        return 0

def process_job_ids(csv_path, service):
    """Process job IDs from CSV and update with email counts."""
    try:
        with open(csv_path, 'r') as file:
            reader = csv.DictReader(file)
            rows = list(reader)
            fieldnames = reader.fieldnames
            
        if 'No_of_emails' not in fieldnames:
            fieldnames.append('No_of_emails')
            for row in rows:
                row['No_of_emails'] = '0'
    
        for row in rows:
            job_id = row.get('Job ID', '').strip() or row.get('Job_ID', '').strip()
            if job_id:
                logging.info(f"Searching for emails with job ID: {job_id}")
                email_count = count_emails_for_job_id(service, job_id)
                row['No_of_emails'] = str(email_count)
                logging.info(f"Found {email_count} emails for {job_id}")
                time.sleep(0.5)  # Rate limiting
    
        with open(csv_path, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
    
        logging.info(f"Successfully updated CSV file: {csv_path}")
        
    except Exception as e:
        logging.error(f"Error processing CSV file: {e}")
        raise

def main():
    """Main function to process both Gmail accounts"""
    try:
        # Initialize results table
        results = []
        
        # Step 1: Authenticate with primary Gmail (extraction account)
        primary_service = auto_authenticate_primary_gmail()
        
        # Step 2: Get recent emails from primary account
        messages = get_recent_emails(primary_service, max_results=25)
        
        if not messages:
            print("No emails found in primary inbox")
            return
        
        # Step 3: Process each email from primary account
        print("\n=== Processing Emails from Primary Account ===")
        for i, message in enumerate(messages, 1):
            message_id = message['id']
            print(f"\nProcessing Email #{i} (ID: {message_id})")
            
            body = extract_email_body(primary_service, message_id)
            
            if body:
                job_details = extract_job_details(body)
                job_details['Email ID'] = message_id  # Temporarily keep for processing
                results.append(job_details)
                
                print(f"Title: {job_details['Title']}")
                print(f"Job_ID: {job_details['Job_ID']}")
                print(f"Due_date: {job_details['Due_date']}")
            else:
                print("No body content could be extracted")
        
        # Create DataFrame and filter rows where Title is missing
        df = pd.DataFrame(results)
        df = df.dropna(subset=['Title'])  # Remove rows with missing Title
        
        if not df.empty:
            csv_path = 'duedates.csv'
            
            # Remove existing CSV file if it exists
            if os.path.exists(csv_path):
                os.remove(csv_path)
                print(f"\nRemoved existing file: '{csv_path}'")
            
            # Remove 'Email ID' column before saving
            df = df.drop(columns=['Email ID'], errors='ignore')
            
            # Save initial CSV (without No_of_emails column yet)
            df.to_csv(csv_path, index=False)
            
            # Step 4: Authenticate with secondary Gmail (counting account)
            secondary_service = auto_authenticate_secondary_gmail()
            
            # Step 5: Process CSV with secondary account to add No_of_emails
            process_job_ids(csv_path, secondary_service)
            
            # Reload the final CSV to display all columns
            final_df = pd.read_csv(csv_path)
            
            # Display final table with all columns (properly formatted)
            print("\n" + "="*50)
            print("FINAL RESULTS".center(50))
            print("="*50)
            
            # Configure pandas display options for better formatting
            pd.set_option('display.max_colwidth', 40)
            pd.set_option('display.width', 120)
            pd.set_option('display.colheader_justify', 'center')
            
            # Print the formatted table
            print(final_df.to_markdown(tablefmt="grid", stralign="left", numalign="left"))
            print("="*50)
            
            print(f"\nResults saved to '{csv_path}'")
        else:
            print("\nNo valid job details found (all rows filtered out)")
            
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


=== Authenticating Primary Gmail Account ===
Found primary token file, loading credentials...
Primary credentials expired, refreshing...


2025-07-23 15:08:25,578 - INFO - file_cache is only supported with oauth2client<4.0.0


Building primary Gmail service...
Primary Gmail service ready!

=== Fetching 25 most recent emails ===
Found 12 emails in inbox

=== Processing Emails from Primary Account ===

Processing Email #1 (ID: 198361443a2622e8)
Title: None
Job_ID: None
Due_date: None

Processing Email #2 (ID: 1983613a0e5db70b)
Title: None
Job_ID: fa1853842
Due_date: None

Processing Email #3 (ID: 19835e64d5ff260b)
Title: None
Job_ID: llC804
Due_date: None

Processing Email #4 (ID: 198322c9c74dd4d2)
Title: None
Job_ID: ea257
Due_date: None

Processing Email #5 (ID: 1982174cf77fb3a3)
Title: Hybrid/Local Analytics Engineer (MS Fabric/12+)
Job_ID: GA-768025
Due_date: 07/16

Processing Email #6 (ID: 198217128c79c421)
Title: Hybrid/Local Privacy Officer/Security Analyst (CISA/CIPM/CIPP)
Job_ID: NC-768095
Due_date: 07/24

Processing Email #7 (ID: 198216d126f86664)
Title: Remote Sitefinity Developer (12+)
Job_ID: NC-765767
Due_date: 06/25

Processing Email #8 (ID: 198216c4da407075)
Title: Remote Sitefinity Developer
J

2025-07-23 15:08:32,180 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-07-23 15:08:32,185 - INFO - Searching for emails with job ID: GA-768025


Building secondary Gmail service...
Secondary Gmail service ready!


2025-07-23 15:08:32,729 - INFO - Found 3 emails for GA-768025
2025-07-23 15:08:33,231 - INFO - Searching for emails with job ID: NC-768095
2025-07-23 15:08:33,663 - INFO - Found 8 emails for NC-768095
2025-07-23 15:08:34,174 - INFO - Searching for emails with job ID: NC-765767
2025-07-23 15:08:34,595 - INFO - Found 6 emails for NC-765767
2025-07-23 15:08:35,124 - INFO - Searching for emails with job ID: NC-765767
2025-07-23 15:08:35,554 - INFO - Found 6 emails for NC-765767
2025-07-23 15:08:36,058 - INFO - Searching for emails with job ID: NC-767681
2025-07-23 15:08:36,494 - INFO - Found 6 emails for NC-767681
2025-07-23 15:08:37,006 - INFO - Searching for emails with job ID: MI-144480
2025-07-23 15:08:37,444 - INFO - Found 14 emails for MI-144480
2025-07-23 15:08:37,956 - INFO - Searching for emails with job ID: MI-144516
2025-07-23 15:08:38,389 - INFO - Found 1 emails for MI-144516
2025-07-23 15:08:38,891 - INFO - Searching for emails with job ID: VA-767445
2025-07-23 15:08:39,326 - 


                  FINAL RESULTS                   
+----+----------------------------------------------------------------+-----------+------------+----------------+
|    | Title                                                          | Job_ID    | Due_date   | No_of_emails   |
+====+================================================================+===========+============+================+
| 0  | Hybrid/Local Analytics Engineer (MS Fabric/12+)                | GA-768025 | 07/16      | 3              |
+----+----------------------------------------------------------------+-----------+------------+----------------+
| 1  | Hybrid/Local Privacy Officer/Security Analyst (CISA/CIPM/CIPP) | NC-768095 | 07/24      | 8              |
+----+----------------------------------------------------------------+-----------+------------+----------------+
| 2  | Remote Sitefinity Developer (12+)                              | NC-765767 | 06/25      | 6              |
+----+------------------------------

In [5]:
import os
import json
import logging
import re
import pandas as pd
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from base64 import urlsafe_b64decode
import pytz
import csv
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def auto_authenticate_primary_gmail():
    """Authenticates with primary Gmail account (token.json)"""
    print("\n=== Authenticating Primary Gmail Account ===")
    creds = None
    token_path = 'token.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        print("Found primary token file, loading credentials...")
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                print("Primary credentials expired, refreshing...")
                creds.refresh(Request())
        except Exception as e:
            print(f"Error loading primary credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        print("No valid primary credentials found, initiating OAuth flow...")
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client.json', SCOPES)
            creds = flow.run_local_server(port=8080)
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
            print("Primary authentication successful! Token saved.")
        except Exception as e:
            print(f"Primary authentication failed: {e}")
            raise
    
    try:
        print("Building primary Gmail service...")
        gmail_service = build('gmail', 'v1', credentials=creds)
        print("Primary Gmail service ready!")
        return gmail_service
    except Exception as e:
        print(f"Failed to build primary Gmail service: {e}")
        raise

def auto_authenticate_secondary_gmail():
    """Authenticates with secondary Gmail account (token1.json)"""
    print("\n=== Authenticating Secondary Gmail Account ===")
    creds = None
    token_path = 'token1.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        print("Found secondary token file, loading credentials...")
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                print("Secondary credentials expired, refreshing...")
                creds.refresh(Request())
        except Exception as e:
            print(f"Error loading secondary credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        print("No valid secondary credentials found, initiating OAuth flow...")
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client1.json', SCOPES)
            creds = flow.run_local_server(port=8081)  # Different port than primary
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
            print("Secondary authentication successful! Token saved.")
        except Exception as e:
            print(f"Secondary authentication failed: {e}")
            raise
    
    try:
        print("Building secondary Gmail service...")
        gmail_service = build('gmail', 'v1', credentials=creds)
        print("Secondary Gmail service ready!")
        return gmail_service
    except Exception as e:
        print(f"Failed to build secondary Gmail service: {e}")
        raise

def get_recent_emails(service, max_results=30):
    """Fetches the most recent emails from the inbox"""
    print(f"\n=== Fetching {max_results} most recent emails ===")
    try:
        results = service.users().messages().list(
            userId="me",
            labelIds=['INBOX'],
            maxResults=max_results
        ).execute()
        messages = results.get('messages', [])
        print(f"Found {len(messages)} emails in inbox")
        return messages
    except Exception as e:
        print(f"Error fetching emails: {e}")
        return []

def decode_base64(data):
    """Decodes base64 email content with proper padding"""
    missing_padding = len(data) % 4
    if missing_padding:
        data += '=' * (4 - missing_padding)
    return urlsafe_b64decode(data)

def extract_email_body(service, message_id):
    """Extracts and decodes the email body content"""
    try:
        msg = service.users().messages().get(
            userId="me",
            id=message_id,
            format='full'
        ).execute()
        
        payload = msg.get('payload', {})
        
        # Extract body from payload
        body = ""
        if 'body' in payload and 'data' in payload['body']:
            body = decode_base64(payload['body']['data']).decode('utf-8', errors='ignore')
        elif 'parts' in payload:
            for part in payload['parts']:
                if part.get('mimeType') in ['text/plain', 'text/html']:
                    if 'body' in part and 'data' in part['body']:
                        body = decode_base64(part['body']['data']).decode('utf-8', errors='ignore')
                        break
                elif 'parts' in part:
                    for subpart in part['parts']:
                        if subpart.get('mimeType') in ['text/plain', 'text/html']:
                            if 'body' in subpart and 'data' in subpart['body']:
                                body = decode_base64(subpart['body']['data']).decode('utf-8', errors='ignore')
                                break
        
        return body if body else None
    except Exception as e:
        print(f"Error processing email {message_id}: {e}")
        return None

def extract_job_details(body):
    """Enhanced version specifically targeting Job ID fields in emails"""
    job_data = {
        'Title': None,
        'Job_ID': None,
        'Due_date': None
    }
    
    if not body:
        return job_data
    
    # Title extraction pattern (unchanged)
    title_pattern = r'(?i)((?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?[^(]*\(.*?\)|(?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?.*?)(?=\s+with\s+|\s*\(|$|\n)'
    title_match = re.search(title_pattern, body)
    if title_match:
        job_data['Title'] = title_match.group(1).strip()
    
    # NEW: Focused Job ID extraction - looks for specific patterns indicating a Job ID field
    job_id_patterns = [
        r'(?i)(?:job\s*|req\s*|position\s*)?(?:id\s*|#\s*|num\s*|number\s*)[:=\s]*([A-Za-z]{2,}-?\d{3,})',  # "Job ID: VA-123456"
        r'(?i)(?:job\s*|req\s*|position\s*)?(?:id\s*|#\s*|num\s*|number\s*)[:=\s]*([A-Za-z]{2,}\s*\d{3,})',  # "Job ID VA 123456"
        r'(?i)\b(?:job\s*|req\s*|position\s*)?(?:id\s*|#\s*|num\s*|number\s*)\b\s*[:-]?\s*([A-Za-z]{2,}-?\d{3,})',  # "Job-ID:VA-123456"
        r'(?<!\w)([A-Za-z]{2,}-?\d{3,})(?!\w)',  # Standalone ID as last resort
    ]
    
    for pattern in job_id_patterns:
        id_match = re.search(pattern, body)
        if id_match:
            job_id = id_match.group(1).strip()
            # Clean up the job ID (remove spaces, normalize format)
            job_id = re.sub(r'\s+', '', job_id)  # Remove any spaces
            job_id = job_id.upper()  # Convert to uppercase for consistency
            job_data['Job_ID'] = job_id
            break
    
    # Date extraction only if we found a Job ID
    if job_data['Job_ID']:
        # Look for dates in parentheses after Job ID
        date_match = re.search(
            r'{}\s*\((\d+)\)'.format(re.escape(job_data['Job_ID'])),
            body
        )
        if date_match:
            full_number = date_match.group(1)
            last_four = full_number[-4:]
            if len(last_four) == 4:
                job_data['Due_date'] = f"{last_four[:2]}/{last_four[2:]}"
    
    return job_data

def count_emails_for_job_id(service, job_id):
    """Count emails containing the job ID in the inbox."""
    try:
        query = f'"{job_id}"'  # Using quotes for exact matching
        results = service.users().messages().list(
            userId="me",
            q=query,
            labelIds=['INBOX']
        ).execute()
        return results.get('resultSizeEstimate', 0)
    except Exception as e:
        logging.error(f"Error counting emails for {job_id}: {e}")
        return 0

def process_job_ids(csv_path, service):
    """Process job IDs from CSV and update with email counts."""
    try:
        with open(csv_path, 'r') as file:
            reader = csv.DictReader(file)
            rows = list(reader)
            fieldnames = reader.fieldnames
            
        if 'No_of_emails' not in fieldnames:
            fieldnames.append('No_of_emails')
            for row in rows:
                row['No_of_emails'] = '0'
    
        for row in rows:
            job_id = row.get('Job ID', '').strip() or row.get('Job_ID', '').strip()
            if job_id:
                logging.info(f"Searching for emails with job ID: {job_id}")
                email_count = count_emails_for_job_id(service, job_id)
                row['No_of_emails'] = str(email_count)
                logging.info(f"Found {email_count} emails for {job_id}")
                time.sleep(0.5)  # Rate limiting
    
        with open(csv_path, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
    
        logging.info(f"Successfully updated CSV file: {csv_path}")
        
    except Exception as e:
        logging.error(f"Error processing CSV file: {e}")
        raise

def main():
    """Main function to process both Gmail accounts"""
    try:
        # Initialize results table
        results = []
        
        # Step 1: Authenticate with primary Gmail (extraction account)
        primary_service = auto_authenticate_primary_gmail()
        
        # Step 2: Get recent emails from primary account
        messages = get_recent_emails(primary_service, max_results=25)
        
        if not messages:
            print("No emails found in primary inbox")
            return
        
        # Step 3: Process each email from primary account
        print("\n=== Processing Emails from Primary Account ===")
        for i, message in enumerate(messages, 1):
            message_id = message['id']
            print(f"\nProcessing Email #{i} (ID: {message_id})")
            
            body = extract_email_body(primary_service, message_id)
            
            if body:
                job_details = extract_job_details(body)
                job_details['Email ID'] = message_id  # Temporarily keep for processing
                results.append(job_details)
                
                print(f"Title: {job_details['Title']}")
                print(f"Job_ID: {job_details['Job_ID']}")
                print(f"Due_date: {job_details['Due_date']}")
            else:
                print("No body content could be extracted")
        
        # Create DataFrame and filter rows where Title is missing
        df = pd.DataFrame(results)
        df = df.dropna(subset=['Title'])  # Remove rows with missing Title
        
        if not df.empty:
            # Remove 'Email ID' column
            df = df.drop(columns=['Email ID'], errors='ignore')
            
            # Create temporary CSV for secondary account processing
            temp_csv = 'temp_duedates.csv'
            df.to_csv(temp_csv, index=False)
            
            # Step 4: Authenticate with secondary Gmail (counting account)
            secondary_service = auto_authenticate_secondary_gmail()
            
            # Step 5: Process CSV with secondary account to add No_of_emails
            process_job_ids(temp_csv, secondary_service)
            
            # Load the final data
            final_df = pd.read_csv(temp_csv)
            
            # Remove temporary CSV
            os.remove(temp_csv)
            
            # Display final table
            print("\n" + "="*100)
            print("FINAL RESULTS".center(100))
            print("="*100)
            
            # Configure display options
            pd.set_option('display.max_colwidth', 40)
            pd.set_option('display.width', 120)
            pd.set_option('display.colheader_justify', 'center')
            
            # Create formatted table
            table = final_df.to_markdown(
                tablefmt="grid",
                stralign="left",
                numalign="left",
                index=False
            )
            
            # Add left margin to each line
            margined_table = [f"    {line}" for line in table.split('\n')]
            print('\n'.join(margined_table))
            print("="*100)
            
            # Save to Excel with formatting
            excel_path = 'duedates_formatted.xlsx'
            try:
                import xlsxwriter
                writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')
                final_df.to_excel(writer, index=False, sheet_name='Job Details')
                
                workbook = writer.book
                worksheet = writer.sheets['Job Details']
                
                # Formatting
                header_format = workbook.add_format({
                    'bold': True,
                    'text_wrap': True,
                    'valign': 'top',
                    'align': 'center',
                    'border': 1
                })
                
                cell_format = workbook.add_format({
                    'text_wrap': True,
                    'valign': 'top',
                    'align': 'left',
                    'border': 1
                })
                
                # Apply formatting
                for col_num, value in enumerate(final_df.columns.values):
                    worksheet.write(0, col_num, value, header_format)
                
                for row in range(1, len(final_df)+1):
                    for col in range(len(final_df.columns)):
                        worksheet.write(row, col, str(final_df.iloc[row-1, col]), cell_format)
                
                # Auto-adjust column widths
                for i, col in enumerate(final_df.columns):
                    max_len = max((
                        final_df[col].astype(str).map(len).max(),
                        len(col)
                    )) + 2
                    worksheet.set_column(i, i, max_len)
                
                writer.close()
                print(f"\nFinal results saved to '{excel_path}'")
                
            except ImportError:
                print("\nError: xlsxwriter not installed - cannot create Excel file")
                print("Install with: pip install xlsxwriter")
                # Fallback to CSV if Excel fails
                csv_path = 'duedates.csv'
                final_df.to_csv(csv_path, index=False)
                print(f"Results saved to '{csv_path}' instead")
            
        else:
            print("\nNo valid job details found (all rows filtered out)")
            
    except Exception as e:
        print(f"An error occurred: {e}")
if __name__ == "__main__":
    main()


=== Authenticating Primary Gmail Account ===
Found primary token file, loading credentials...
Primary credentials expired, refreshing...


2025-07-24 12:19:24,169 - INFO - file_cache is only supported with oauth2client<4.0.0


Building primary Gmail service...
Primary Gmail service ready!

=== Fetching 25 most recent emails ===
Found 22 emails in inbox

=== Processing Emails from Primary Account ===

Processing Email #1 (ID: 198393575cacd4f9)
Title: Onsite/Local Compliance Officer
Job_ID: FL-773154
Due_date: 07/29

Processing Email #2 (ID: 198392a579360a53)
Title: Onsite/Local Compliance Officer
Job_ID: FL-773156
Due_date: 07/29

Processing Email #3 (ID: 198392635986247c)
Title: Hybrid/Local .NET Developer (10+)
Job_ID: SC-7571
Due_date: 07/30

Processing Email #4 (ID: 1983917093cec522)
Title: Onsite/Local Compliance Officer
Job_ID: FL-773155
Due_date: 07/29

Processing Email #5 (ID: 19838198f3582674)
Title: Hybrid/Local Security Architect (CISSP/CCSP/Azure/AWS/GCP/15+)
Job_ID: VA-767227
Due_date: 07/28

Processing Email #6 (ID: 19837c6c0c189722)
Title: Hybrid/Local .NET/C# Developer (15+)
Job_ID: VA-768268
Due_date: 07/28

Processing Email #7 (ID: 19837c14ea90518f)
Title: Onsite/Local Desktop Support Techni

2025-07-24 12:19:36,491 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-07-24 12:19:36,498 - INFO - Searching for emails with job ID: FL-773154


Building secondary Gmail service...
Secondary Gmail service ready!


2025-07-24 12:19:37,033 - INFO - Found 0 emails for FL-773154
2025-07-24 12:19:37,547 - INFO - Searching for emails with job ID: FL-773156
2025-07-24 12:19:37,966 - INFO - Found 0 emails for FL-773156
2025-07-24 12:19:38,467 - INFO - Searching for emails with job ID: SC-7571
2025-07-24 12:19:38,903 - INFO - Found 0 emails for SC-7571
2025-07-24 12:19:39,407 - INFO - Searching for emails with job ID: FL-773155
2025-07-24 12:19:39,829 - INFO - Found 0 emails for FL-773155
2025-07-24 12:19:40,344 - INFO - Searching for emails with job ID: VA-767227
2025-07-24 12:19:40,808 - INFO - Found 4 emails for VA-767227
2025-07-24 12:19:41,317 - INFO - Searching for emails with job ID: VA-768268
2025-07-24 12:19:41,755 - INFO - Found 14 emails for VA-768268
2025-07-24 12:19:42,260 - INFO - Searching for emails with job ID: VA-768094
2025-07-24 12:19:42,690 - INFO - Found 0 emails for VA-768094
2025-07-24 12:19:43,203 - INFO - Searching for emails with job ID: NC-764016
2025-07-24 12:19:43,644 - INFO


                                           FINAL RESULTS                                            
    +----------------------------------------------------------------+-----------+------------+----------------+
    | Title                                                          | Job_ID    | Due_date   | No_of_emails   |
    +================================================================+===========+============+================+
    | Onsite/Local Compliance Officer                                | FL-773154 | 07/29      | 0              |
    +----------------------------------------------------------------+-----------+------------+----------------+
    | Onsite/Local Compliance Officer                                | FL-773156 | 07/29      | 0              |
    +----------------------------------------------------------------+-----------+------------+----------------+
    | Hybrid/Local .NET Developer (10+)                              | SC-7571   | 07/30      | 0          

In [3]:
import os
import json
import logging
import re
import pandas as pd
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from base64 import urlsafe_b64decode
import pytz
import csv
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def auto_authenticate_primary_gmail():
    """Authenticates with primary Gmail account (token.json)"""
    print("\n=== Authenticating Primary Gmail Account ===")
    creds = None
    token_path = 'token.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        print("Found primary token file, loading credentials...")
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                print("Primary credentials expired, refreshing...")
                creds.refresh(Request())
        except Exception as e:
            print(f"Error loading primary credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        print("No valid primary credentials found, initiating OAuth flow...")
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client.json', SCOPES)
            creds = flow.run_local_server(port=8080)
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
            print("Primary authentication successful! Token saved.")
        except Exception as e:
            print(f"Primary authentication failed: {e}")
            raise
    
    try:
        print("Building primary Gmail service...")
        gmail_service = build('gmail', 'v1', credentials=creds)
        print("Primary Gmail service ready!")
        return gmail_service
    except Exception as e:
        print(f"Failed to build primary Gmail service: {e}")
        raise

def auto_authenticate_secondary_gmail():
    """Authenticates with secondary Gmail account (token1.json)"""
    print("\n=== Authenticating Secondary Gmail Account ===")
    creds = None
    token_path = 'token1.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    
    if os.path.exists(token_path):
        print("Found secondary token file, loading credentials...")
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
            if creds.expired and creds.refresh_token:
                print("Secondary credentials expired, refreshing...")
                creds.refresh(Request())
        except Exception as e:
            print(f"Error loading secondary credentials: {e}")
            creds = None
    
    if not creds or not creds.valid:
        print("No valid secondary credentials found, initiating OAuth flow...")
        try:
            flow = InstalledAppFlow.from_client_secrets_file('client1.json', SCOPES)
            creds = flow.run_local_server(port=8081)
            token_data = json.loads(creds.to_json())
            token_data['creation_time'] = datetime.now(pytz.UTC).isoformat()
            with open(token_path, 'w') as token:
                json.dump(token_data, token)
            print("Secondary authentication successful! Token saved.")
        except Exception as e:
            print(f"Secondary authentication failed: {e}")
            raise
    
    try:
        print("Building secondary Gmail service...")
        gmail_service = build('gmail', 'v1', credentials=creds)
        print("Secondary Gmail service ready!")
        return gmail_service
    except Exception as e:
        print(f"Failed to build secondary Gmail service: {e}")
        raise

def get_recent_emails(service, max_results=30):
    """Fetches the most recent emails from the inbox"""
    print(f"\n=== Fetching {max_results} most recent emails ===")
    try:
        results = service.users().messages().list(
            userId="me",
            labelIds=['INBOX'],
            maxResults=max_results
        ).execute()
        messages = results.get('messages', [])
        print(f"Found {len(messages)} emails in inbox")
        return messages
    except Exception as e:
        print(f"Error fetching emails: {e}")
        return []

def decode_base64(data):
    """Decodes base64 email content with proper padding"""
    missing_padding = len(data) % 4
    if missing_padding:
        data += '=' * (4 - missing_padding)
    return urlsafe_b64decode(data)

def extract_email_body(service, message_id):
    """Extracts and decodes the email body content"""
    try:
        msg = service.users().messages().get(
            userId="me",
            id=message_id,
            format='full'
        ).execute()
        
        payload = msg.get('payload', {})
        
        # Extract body from payload
        body = ""
        if 'body' in payload and 'data' in payload['body']:
            body = decode_base64(payload['body']['data']).decode('utf-8', errors='ignore')
        elif 'parts' in payload:
            for part in payload['parts']:
                if part.get('mimeType') in ['text/plain', 'text/html']:
                    if 'body' in part and 'data' in part['body']:
                        body = decode_base64(part['body']['data']).decode('utf-8', errors='ignore')
                        break
                elif 'parts' in part:
                    for subpart in part['parts']:
                        if subpart.get('mimeType') in ['text/plain', 'text/html']:
                            if 'body' in subpart and 'data' in subpart['body']:
                                body = decode_base64(subpart['body']['data']).decode('utf-8', errors='ignore')
                                break
        
        return body if body else None
    except Exception as e:
        print(f"Error processing email {message_id}: {e}")
        return None

def extract_job_details(body):
    """Enhanced version specifically targeting Job ID fields in emails"""
    job_data = {
        'Title': None,
        'Job_ID': None,
        'Due_date': None
    }
    
    if not body:
        return job_data
    
    # Title extraction pattern (unchanged)
    title_pattern = r'(?i)((?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?[^(]*\(.*?\)|(?:Hybrid|Onsite|Remote)(?:\s*/\s*Local)?.*?)(?=\s+with\s+|\s*\(|$|\n)'
    title_match = re.search(title_pattern, body)
    if title_match:
        job_data['Title'] = title_match.group(1).strip()
    
    # NEW: Focused Job ID extraction - looks for specific patterns indicating a Job ID field
    job_id_patterns = [
        r'(?i)(?:job\s*|req\s*|position\s*)?(?:id\s*|#\s*|num\s*|number\s*)[:=\s]*([A-Za-z]{2,}-?\d{3,})',  # "Job ID: VA-123456"
        r'(?i)(?:job\s*|req\s*|position\s*)?(?:id\s*|#\s*|num\s*|number\s*)[:=\s]*([A-Za-z]{2,}\s*\d{3,})',  # "Job ID VA 123456"
        r'(?i)\b(?:job\s*|req\s*|position\s*)?(?:id\s*|#\s*|num\s*|number\s*)\b\s*[:-]?\s*([A-Za-z]{2,}-?\d{3,})',  # "Job-ID:VA-123456"
        r'(?<!\w)([A-Za-z]{2,}-?\d{3,})(?!\w)',  # Standalone ID as last resort
    ]
    
    for pattern in job_id_patterns:
        id_match = re.search(pattern, body)
        if id_match:
            job_id = id_match.group(1).strip()
            # Clean up the job ID (remove spaces, normalize format)
            job_id = re.sub(r'\s+', '', job_id)  # Remove any spaces
            job_id = job_id.upper()  # Convert to uppercase for consistency
            job_data['Job_ID'] = job_id
            break
    
    # Date extraction only if we found a Job ID
    if job_data['Job_ID']:
        # Look for dates in parentheses after Job ID
        date_match = re.search(
            r'{}\s*\((\d+)\)'.format(re.escape(job_data['Job_ID'])),
            body
        )
        if date_match:
            full_number = date_match.group(1)
            last_four = full_number[-4:]
            if len(last_four) == 4:
                job_data['Due_date'] = f"{last_four[:2]}/{last_four[2:]}"
    
    return job_data

def count_emails_for_job_id(service, job_id):
    """Count emails containing the job ID in the inbox."""
    try:
        query = f'"{job_id}"'  # Using quotes for exact matching
        results = service.users().messages().list(
            userId="me",
            q=query,
            labelIds=['INBOX']
        ).execute()
        return results.get('resultSizeEstimate', 0)
    except Exception as e:
        logging.error(f"Error counting emails for {job_id}: {e}")
        return 0

def process_job_ids(csv_path, service):
    """Process job IDs from CSV and update with email counts."""
    try:
        with open(csv_path, 'r') as file:
            reader = csv.DictReader(file)
            rows = list(reader)
            fieldnames = reader.fieldnames
            
        if 'No_of_emails' not in fieldnames:
            fieldnames.append('No_of_emails')
            for row in rows:
                row['No_of_emails'] = '0'
    
        for row in rows:
            job_id = row.get('Job ID', '').strip() or row.get('Job_ID', '').strip()
            if job_id:
                logging.info(f"Searching for emails with job ID: {job_id}")
                email_count = count_emails_for_job_id(service, job_id)
                row['No_of_emails'] = str(email_count)
                logging.info(f"Found {email_count} emails for {job_id}")
                time.sleep(0.5)  # Rate limiting
    
        with open(csv_path, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
    
        logging.info(f"Successfully updated CSV file: {csv_path}")
        
    except Exception as e:
        logging.error(f"Error processing CSV file: {e}")
        raise

def send_results_email(excel_path, recipient_email):
    """Send the results Excel file as an email attachment"""
    try:
        # Email configuration - REPLACE THESE WITH YOUR DETAILS
        sender_email = "gorintalakavya@gmail.com"
        password = "tihr qpwm pwwv dtzf"  # Use App Password if 2FA is enabled
        smtp_server = "smtp.gmail.com"
        smtp_port = 587
        
        # Create message container
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = "Job Application Tracker Results"
        
        # Email body
        body = f"""Please find attached the latest job application tracking results.
        
        Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        
        This file contains:
        - Job Titles
        - Job IDs
        - Due Dates
        - Email Counts
        """
        msg.attach(MIMEText(body, 'plain'))
        
        # Attach Excel file
        with open(excel_path, 'rb') as file:
            part = MIMEApplication(file.read(), Name="Job_Tracker_Report.xlsx")
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(excel_path)}"'
        msg.attach(part)
        
        # Send email
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, password)
            server.send_message(msg)
        
        print(f"\nResults successfully emailed to {recipient_email}")
        
    except Exception as e:
        print(f"\nFailed to send email: {str(e)}")

def main():
    """Main function to process both Gmail accounts"""
    try:
        # Initialize results table
        results = []
        
        # Step 1: Authenticate with primary Gmail (extraction account)
        primary_service = auto_authenticate_primary_gmail()
        
        # Step 2: Get recent emails from primary account
        messages = get_recent_emails(primary_service, max_results=25)
        
        if not messages:
            print("No emails found in primary inbox")
            return
        
        # Step 3: Process each email from primary account
        print("\n=== Processing Emails from Primary Account ===")
        for i, message in enumerate(messages, 1):
            message_id = message['id']
            print(f"\nProcessing Email #{i} (ID: {message_id})")
            
            body = extract_email_body(primary_service, message_id)
            
            if body:
                job_details = extract_job_details(body)
                job_details['Email ID'] = message_id  # Temporarily keep for processing
                results.append(job_details)
                
                print(f"Title: {job_details['Title']}")
                print(f"Job_ID: {job_details['Job_ID']}")
                print(f"Due_date: {job_details['Due_date']}")
            else:
                print("No body content could be extracted")
        
        # Create DataFrame and filter rows where Title is missing
        df = pd.DataFrame(results)
        df = df.dropna(subset=['Title'])  # Remove rows with missing Title
        
        if not df.empty:
            # Create temporary CSV for secondary account processing
            temp_csv = 'temp_duedates.csv'
            df = df.drop(columns=['Email ID'], errors='ignore')
            df.to_csv(temp_csv, index=False)
            
            # Step 4: Authenticate with secondary Gmail (counting account)
            secondary_service = auto_authenticate_secondary_gmail()
            
            # Step 5: Process CSV with secondary account to add No_of_emails
            process_job_ids(temp_csv, secondary_service)
            
            # Load the final data
            final_df = pd.read_csv(temp_csv)
            
            # Remove temporary CSV
            os.remove(temp_csv)
            
            # Display final table
            print("\n" + "="*100)
            print("FINAL RESULTS".center(100))
            print("="*100)
            
            # Configure display options
            pd.set_option('display.max_colwidth', 40)
            pd.set_option('display.width', 120)
            pd.set_option('display.colheader_justify', 'center')
            
            # Create formatted table
            table = final_df.to_markdown(
                tablefmt="grid",
                stralign="left",
                numalign="left",
                index=False
            )
            
            # Add left margin to each line
            margined_table = [f"    {line}" for line in table.split('\n')]
            print('\n'.join(margined_table))
            print("="*100)
            
            # Save to Excel with formatting
            excel_path = 'duedates_formatted.xlsx'
            try:
                import xlsxwriter
                writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')
                final_df.to_excel(writer, index=False, sheet_name='Job Details')
                
                workbook = writer.book
                worksheet = writer.sheets['Job Details']
                
                # Formatting
                header_format = workbook.add_format({
                    'bold': True,
                    'text_wrap': True,
                    'valign': 'top',
                    'align': 'center',
                    'border': 1
                })
                
                cell_format = workbook.add_format({
                    'text_wrap': True,
                    'valign': 'top',
                    'align': 'left',
                    'border': 1
                })
                
                # Apply formatting
                for col_num, value in enumerate(final_df.columns.values):
                    worksheet.write(0, col_num, value, header_format)
                
                for row in range(1, len(final_df)+1):
                    for col in range(len(final_df.columns)):
                        worksheet.write(row, col, str(final_df.iloc[row-1, col]), cell_format)
                
                # Auto-adjust column widths
                for i, col in enumerate(final_df.columns):
                    max_len = max((
                        final_df[col].astype(str).map(len).max(),
                        len(col)
                    )) + 2
                    worksheet.set_column(i, i, max_len)
                
                writer.close()
                print(f"\nFinal results saved to '{excel_path}'")
                  
                # Send email with results - REPLACE WITH ACTUAL RECIPIENT
                send_results_email(excel_path, "nandini.ka12345@gmail.com")
                
            except ImportError:
                print("\nError: xlsxwriter not installed - cannot create Excel file")
                print("Install with: pip install xlsxwriter")
                # Fallback to CSV if Excel fails
                csv_path = 'duedates.csv'
                final_df.to_csv(csv_path, index=False)
                print(f"Results saved to '{csv_path}'")
            
        else:
            print("\nNo valid job details found (all rows filtered out)")
            
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


=== Authenticating Primary Gmail Account ===
Found primary token file, loading credentials...
Primary credentials expired, refreshing...


2025-07-24 12:07:14,996 - INFO - file_cache is only supported with oauth2client<4.0.0


Building primary Gmail service...
Primary Gmail service ready!

=== Fetching 25 most recent emails ===
Found 22 emails in inbox

=== Processing Emails from Primary Account ===

Processing Email #1 (ID: 198393575cacd4f9)
Title: Onsite/Local Compliance Officer
Job_ID: FL-773154
Due_date: 07/29

Processing Email #2 (ID: 198392a579360a53)
Title: Onsite/Local Compliance Officer
Job_ID: FL-773156
Due_date: 07/29

Processing Email #3 (ID: 198392635986247c)
Title: Hybrid/Local .NET Developer (10+)
Job_ID: SC-7571
Due_date: 07/30

Processing Email #4 (ID: 1983917093cec522)
Title: Onsite/Local Compliance Officer
Job_ID: FL-773155
Due_date: 07/29

Processing Email #5 (ID: 19838198f3582674)
Title: Hybrid/Local Security Architect (CISSP/CCSP/Azure/AWS/GCP/15+)
Job_ID: VA-767227
Due_date: 07/28

Processing Email #6 (ID: 19837c6c0c189722)
Title: Hybrid/Local .NET/C# Developer (15+)
Job_ID: VA-768268
Due_date: 07/28

Processing Email #7 (ID: 19837c14ea90518f)
Title: Onsite/Local Desktop Support Techni

2025-07-24 12:07:27,609 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-07-24 12:07:27,627 - INFO - Searching for emails with job ID: FL-773154


Building secondary Gmail service...
Secondary Gmail service ready!


2025-07-24 12:07:28,223 - INFO - Found 0 emails for FL-773154
2025-07-24 12:07:28,738 - INFO - Searching for emails with job ID: FL-773156
2025-07-24 12:07:29,169 - INFO - Found 0 emails for FL-773156
2025-07-24 12:07:29,676 - INFO - Searching for emails with job ID: SC-7571
2025-07-24 12:07:30,110 - INFO - Found 0 emails for SC-7571
2025-07-24 12:07:30,621 - INFO - Searching for emails with job ID: FL-773155
2025-07-24 12:07:31,061 - INFO - Found 0 emails for FL-773155
2025-07-24 12:07:31,576 - INFO - Searching for emails with job ID: VA-767227
2025-07-24 12:07:32,135 - INFO - Found 4 emails for VA-767227
2025-07-24 12:07:32,647 - INFO - Searching for emails with job ID: VA-768268
2025-07-24 12:07:33,149 - INFO - Found 14 emails for VA-768268
2025-07-24 12:07:33,652 - INFO - Searching for emails with job ID: VA-768094
2025-07-24 12:07:34,110 - INFO - Found 0 emails for VA-768094
2025-07-24 12:07:34,618 - INFO - Searching for emails with job ID: NC-764016
2025-07-24 12:07:35,196 - INFO


                                           FINAL RESULTS                                            
    +----------------------------------------------------------------+-----------+------------+----------------+
    | Title                                                          | Job_ID    | Due_date   | No_of_emails   |
    +================================================================+===========+============+================+
    | Onsite/Local Compliance Officer                                | FL-773154 | 07/29      | 0              |
    +----------------------------------------------------------------+-----------+------------+----------------+
    | Onsite/Local Compliance Officer                                | FL-773156 | 07/29      | 0              |
    +----------------------------------------------------------------+-----------+------------+----------------+
    | Hybrid/Local .NET Developer (10+)                              | SC-7571   | 07/30      | 0          